# Update config json file with translated names for the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

The scripts described in this document are used to:

- Check translated names in the database
- Read/write json file with information for each EFG

## Set-up
Load all the libraries we will need in this script:

In [1]:
import os
from pathlib import Path
from configparser import ConfigParser
import psycopg2
import pandas as pd
import json
import requests
import re


Read configuration parameters for the connection to the current version of the database:

In [2]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [3]:
conn = psycopg2.connect(**params)
cur = conn.cursor()

## List of EFG names

In [4]:
qry="""
SELECT code, t.language, f.name, t.name
FROM  efg_translations t
LEFT JOIN functional_groups f
USING (code)
WHERE language = 'es'
""";
cur.execute(qry)
efg_names=cur.fetchall()

In [5]:

df=pd.DataFrame(efg_names,columns=['code','language', 'name_orig', 'name_trans'])


In [6]:
df1=df.query("code=='T7.5'")
df1

,code,language,name_orig,name_trans
102,T7.5,es,T7.5 Derived semi-natural pastures and old fields,"T7.5 Campos seminaturales, modificados y aband..."


## List of biome names

In [7]:
qry="""
SELECT biome_code, t.language, f.name, t.name
FROM  biome_translations t
LEFT JOIN biomes f
USING (biome_code)
WHERE language = 'es'
""";
cur.execute(qry)
biome_names=cur.fetchall()

In [8]:

df2=pd.DataFrame(biome_names,columns=['code','language', 'name_orig', 'name_trans'])
df2

,code,language,name_orig,name_trans
0,F1,es,F1. Rivers and streams biome,F1. Bioma de ríos y arroyos
1,F2,es,F2. Lakes biome,F2. Bioma de lagos
2,F3,es,F3. Artificial wetlands biome,F3. Bioma de humedales artificiales
3,FM1,es,FM1. Semi-confined transitional waters biome,FM1. Bioma de aguas de transición semiconfinadas
4,M1,es,M1. Marine shelf biome,M1. Bioma de plataforma marina
5,M2,es,M2. Pelagic ocean waters biome,M2. Bioma de aguas oceánicas pelágicas
6,M3,es,M3. Deep sea floors biome,M3. Bioma de los fondos marinos profundos
7,M4,es,M4. Anthropogenic marine biome,M4. Bioma marino antropogénico
8,MFT1,es,MFT1. Brackish tidal biome,MFT1. Bioma de mareas salobres
9,MT1,es,MT1. Shorelines biome,MT1. Bioma de las costas


## Read json files

Read json file

In [9]:
#config.json <- read_json(sprintf("%s/data/config/groups.json", Sys.getenv("WEBDATAREPO")))
home = os.environ.get('HOME')
webdatarepo = os.environ.get('WEBDATAREPO', home + '/proyectos/typology-website/typology-map-data')


In [10]:
Path(webdatarepo)

PosixPath('/Users/z3529065/proyectos/typology-website/typology-map-data')

In [11]:
web_config_groups = Path(webdatarepo) / 'data/config/groups.json'
with open(web_config_groups) as f:
    groups_json = json.load(f)

web_config_biomes = Path(webdatarepo) / 'data/config/biomes.json'
with open(web_config_biomes) as f:
    biome_json = json.load(f)




In [12]:
biome_json[10]

{'id': 'MT2',
 'path': 'mt_2',
 'realm': 'MT',
 'title': {'en': 'Supralittoral coastal biome',
  'es': 'Bioma costero supralitoral'},
 'image': {'credit': {'en': 'Jo Hiscock, New Zealand Department of Conservation, CC BY 4.0'},
  'caption': {'en': 'Auckland Islands sea cliff heath and rookery'}}}

In [13]:
for r in groups_json:
    efg=r['id']
    if 'title' in r.keys():
        nombre = df.loc[df['code'] == efg, 'name_trans'].item()
        r['title']['es']=re.sub(r"^[TSFM0-9\.]+ ", "", nombre)

In [14]:
for r in biome_json:
    efg=r['id']
    if 'title' in r.keys():
        nombre = df2.loc[df2['code'] == efg, 'name_trans'].item()
        r['title']['es']=re.sub(r"^[TSFM0-9\.]+ ", "", nombre)

In [15]:
groups_json[100]
                

{'id': 'T7.3',
 'path': 't_7_3',
 'biome': 'T7',
 'layer': {'source': 'mapbox',
  'type': 'raster',
  'tileset': 'jrfep.T7_3_web_alt',
  'extent': {'N': '90', 'S': '90', 'W': '180', 'E': '180'}},
 'title': {'en': 'Plantations', 'es': 'Plantaciones'},
 'image': {'credit': {'en': 'Prashant Y on Flickr, CC BY 2.0'},
  'caption': {'en': 'Multi-species plantation (shade coffee), Chikmagalur, India'}}}

In [16]:
biome_json[10]

{'id': 'MT2',
 'path': 'mt_2',
 'realm': 'MT',
 'title': {'en': 'Supralittoral coastal biome',
  'es': 'Bioma costero supralitoral'},
 'image': {'credit': {'en': 'Jo Hiscock, New Zealand Department of Conservation, CC BY 4.0'},
  'caption': {'en': 'Auckland Islands sea cliff heath and rookery'}}}

In [17]:
with open(web_config_groups,"w", encoding='utf8') as f:
    json.dump(groups_json, f, ensure_ascii=False, indent = 2)


In [18]:
with open(web_config_biomes,"w", encoding='utf8') as f:
    json.dump(biome_json, f, ensure_ascii=False, indent = 2)


## That's it

Let's close the connection:

In [19]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
